<a href="https://colab.research.google.com/github/antoniocfetngnu/InteligArtificial1/blob/main/redesNeuronales/lab05_RedesNeuronales_Caldeorn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Descripcion Dataset

Este conjunto de datos resume un conjunto heterogéneo de características sobre artículos publicados por Mashable en un período de dos años. El objetivo es predecir el número de veces que se comparten en redes sociales (popularidad).

###Adaptacion Salida (Clasificacion)
En este conjunto de datos, la columna "shares" se considera como la etiqueta (o salida) que se busca predecir. Para facilitar la clasificación, se ha adaptado la columna de "shares" a tres categorías de salida: 0, 1 y 2. Estas categorías corresponden a la cantidad de veces que se comparten en redes sociales, dividiendo el valor original de "shares" entre la media y luego dividiendo entre 2 para obtener tres categorías distintas.

###Importacion de librerias

In [1]:
import os
import pandas as pd
import numpy as np
from matplotlib import pyplot
from scipy import optimize
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import random
# tells matplotlib to embed plots within the notebook
%matplotlib inline

##Preparación del Dataset
###Importacion

In [2]:
df = pd.read_csv('OnlineNewsPopularity.csv')

In [3]:
pd.set_option('display.max_columns', None)
print(df.describe())


          timedelta   n_tokens_title   n_tokens_content   n_unique_tokens  \
count  39644.000000     39644.000000       39644.000000      39644.000000   
mean     354.530471        10.398749         546.514731          0.548216   
std      214.163767         2.114037         471.107508          3.520708   
min        8.000000         2.000000           0.000000          0.000000   
25%      164.000000         9.000000         246.000000          0.470870   
50%      339.000000        10.000000         409.000000          0.539226   
75%      542.000000        12.000000         716.000000          0.608696   
max      731.000000        23.000000        8474.000000        701.000000   

        n_non_stop_words   n_non_stop_unique_tokens     num_hrefs  \
count       39644.000000               39644.000000  39644.000000   
mean            0.996469                   0.689175     10.883690   
std             5.231231                   3.264816     11.332017   
min             0.000000      

In [4]:
print(df.head())

                                                 url   timedelta  \
0  http://mashable.com/2013/01/07/amazon-instant-...       731.0   
1  http://mashable.com/2013/01/07/ap-samsung-spon...       731.0   
2  http://mashable.com/2013/01/07/apple-40-billio...       731.0   
3  http://mashable.com/2013/01/07/astronaut-notre...       731.0   
4   http://mashable.com/2013/01/07/att-u-verse-apps/       731.0   

    n_tokens_title   n_tokens_content   n_unique_tokens   n_non_stop_words  \
0             12.0              219.0          0.663594                1.0   
1              9.0              255.0          0.604743                1.0   
2              9.0              211.0          0.575130                1.0   
3              9.0              531.0          0.503788                1.0   
4             13.0             1072.0          0.415646                1.0   

    n_non_stop_unique_tokens   num_hrefs   num_self_hrefs   num_imgs  \
0                   0.815385         4.0          

Verifiacion del valor maximo y minimo en la columna Shares para hacer una correcta adaptación

In [5]:
valor_maximo = df[' shares'].max()
valor_minimo = df[' shares'].min()

print(f"Valor máximo: {valor_maximo}")
print(f"Valor mínimo: {valor_minimo}")

Valor máximo: 843300
Valor mínimo: 1


Adaptación de la salida en 3 valores de clasificacion 0,1,2.
Los "Shares" indican la popularidad de un artículo publicado según la cantidad de compartidos. Por tanto en la adaptacion se tomará 3 categorías
No Popular=0, Neutro=1, Popular=2

In [6]:
media_shares = df[' shares'].mean()

# Divide los valores en 3 categorías basadas en la media
df['categoria_shares'] = pd.cut(df[' shares'], bins=[0, media_shares/2, media_shares, float('inf')], labels=[0, 1, 2], right=False)

# Verificar los resultados
cantidad_por_categoria = df['categoria_shares'].value_counts()
print("Cantidad por categoría actualizada:")
print(cantidad_por_categoria)
print("Categorias:\n0: Desde 0 a ",media_shares/2,"\n1: Desde ",media_shares/2," hasta ",media_shares,"\n2: Desde ",media_shares, " en adelante")


Cantidad por categoría actualizada:
categoria_shares
0    22542
1     9023
2     8079
Name: count, dtype: int64
Categorias:
0: Desde 0 a  1697.6900918171727 
1: Desde  1697.6900918171727  hasta  3395.3801836343455 
2: Desde  3395.3801836343455  en adelante


Comprobacion de tipos de datos mixtos en una misma columna

In [7]:
# Obtiene información sobre los tipos de datos de cada columna
column_types = df.dtypes

# Inicializa una lista para almacenar las columnas con tipos de datos mixtos
mixed_type_columns = []

# Itera sobre las columnas y verifica si cada una tiene más de un tipo de dato
for column in column_types.index:
    unique_types = df[column].apply(type).unique()
    if len(unique_types) > 1:
        mixed_type_columns.append(column)

# Imprime las columnas con tipos de datos mixtos
print("Columnas con tipos de datos mixtos:")
print(mixed_type_columns)

Columnas con tipos de datos mixtos:
[]


Comprobacion de presencia de valores nulos (en este caso no hay nulos)

In [8]:
# Contar los valores nulos en cada columna del DataFrame
valores_nulos_por_columna = df.isna().sum()
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
# Mostrar el resultado
print("Valores nulos por columna:")
print(valores_nulos_por_columna)

Valores nulos por columna:
url                               0
 timedelta                        0
 n_tokens_title                   0
 n_tokens_content                 0
 n_unique_tokens                  0
 n_non_stop_words                 0
 n_non_stop_unique_tokens         0
 num_hrefs                        0
 num_self_hrefs                   0
 num_imgs                         0
 num_videos                       0
 average_token_length             0
 num_keywords                     0
 data_channel_is_lifestyle        0
 data_channel_is_entertainment    0
 data_channel_is_bus              0
 data_channel_is_socmed           0
 data_channel_is_tech             0
 data_channel_is_world            0
 kw_min_min                       0
 kw_max_min                       0
 kw_avg_min                       0
 kw_min_max                       0
 kw_max_max                       0
 kw_avg_max                       0
 kw_min_avg                       0
 kw_max_avg                       0
 

Eliminacion de columnas innecesarias

In [9]:
df = df.drop('url', axis=1)

In [10]:
df = df.dropna()

In [11]:
# Restablecer las opciones de visualización a sus valores por defecto
pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')

In [12]:
df['categoria_shares'] = df['categoria_shares'].astype(int)

# Verificar el cambio
print(df['categoria_shares'].head(10))

0    0
1    0
2    0
3    0
4    0
5    0
6    0
7    0
8    2
9    0
Name: categoria_shares, dtype: int64


In [13]:
df = df.drop(' shares', axis=1)

In [14]:
print(df.describe())

          timedelta   n_tokens_title   n_tokens_content   n_unique_tokens  \
count  39644.000000     39644.000000       39644.000000      39644.000000   
mean     354.530471        10.398749         546.514731          0.548216   
std      214.163767         2.114037         471.107508          3.520708   
min        8.000000         2.000000           0.000000          0.000000   
25%      164.000000         9.000000         246.000000          0.470870   
50%      339.000000        10.000000         409.000000          0.539226   
75%      542.000000        12.000000         716.000000          0.608696   
max      731.000000        23.000000        8474.000000        701.000000   

        n_non_stop_words   n_non_stop_unique_tokens     num_hrefs  \
count       39644.000000               39644.000000  39644.000000   
mean            0.996469                   0.689175     10.883690   
std             5.231231                   3.264816     11.332017   
min             0.000000      

#Separación en entrenamiento y prueba


In [15]:
X = df.drop(columns=['categoria_shares'])  # Características, se eliminan las columnas de etiquetas
y = df['categoria_shares']  # Etiquetas

# Verificar las dimensiones de X y y
print("Dimensiones de X:", X.shape)
print("Dimensiones de y:", y.shape)

Dimensiones de X: (39644, 59)
Dimensiones de y: (39644,)


In [16]:
print(y.value_counts())

categoria_shares
0    22542
1     9023
2     8079
Name: count, dtype: int64


In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [18]:
# Verificar la proporción de clases en los conjuntos de entrenamiento y prueba
print("Proporción de clases en y_train:")
print(y_train.value_counts(normalize=True))

print("\nProporción de clases en y_test:")
print(y_test.value_counts(normalize=True))

Proporción de clases en y_train:
categoria_shares
0    0.568627
1    0.227589
2    0.203784
Name: proportion, dtype: float64

Proporción de clases en y_test:
categoria_shares
0    0.568546
1    0.227645
2    0.203809
Name: proportion, dtype: float64


In [19]:
# 1. Ajustar input_layer_size al número de características en X_train
input_layer_size = X_train.shape[1]

# 2. Ajustar num_labels al número de categorías en y_train
num_labels = len(np.unique(y_train))

###Normalizacion

In [20]:
# Normalizar características solo en los conjuntos de entrenamiento y prueba de X (caracteristicas)
scaler = StandardScaler()
X = scaler.fit_transform(X_train)
Xp = scaler.transform(X_test)
print(X)
print('-'*30)
print(Xp)
m, n = X.shape
print('valor de m:',m)
print('valor de n:',n)

y = y_train.values
yp = y_test.values
# Verificar las dimensiones de X_train_np y y_train_np
print("Dimensiones de X_train_np:", X.shape)
print("Dimensiones de y_train_np:", y.shape)

[[ 0.57755307  0.75974389 -0.30732414 ...  1.23685855 -0.22459879
   1.07553011]
 [-1.35468875  0.75974389  0.48013696 ... -0.26696477  0.83678
  -0.68838021]
 [ 0.31555418  0.75974389  1.57079164 ... -0.26696477  0.83678
  -0.68838021]
 ...
 [-1.08801131 -1.13718598 -0.86317903 ... -0.26696477  0.83678
  -0.68838021]
 [-0.05872995 -1.61141845 -0.83159636 ... -0.26696477  0.83678
  -0.68838021]
 [-1.2049751  -1.61141845 -0.79580268 ... -0.26696477 -0.93218465
  -0.68838021]]
------------------------------
[[ 1.27933582 -0.66295351 -0.51366421 ...  2.55270396 -0.48994349
   2.61895164]
 [ 0.79276645  0.75974389 -0.07571793 ...  0.26563932 -0.22459879
  -0.06366198]
 [-1.27047482  2.65667376 -0.69684366 ...  0.10899106  0.83678
  -0.24740263]
 ...
 [ 0.9518372  -0.18872105 -0.58104056 ...  0.48494689 -0.22459879
   0.19357495]
 [-1.33597454 -1.13718598 -0.68631611 ... -0.26696477  0.83678
  -0.68838021]
 [-1.34533165  0.75974389  1.11179025 ... -0.26696477  0.83678
  -0.68838021]]
valor 

In [21]:
print("Input Layer Size:",input_layer_size)
print("num Labels:",num_labels)

Input Layer Size: 59
num Labels: 3


In [22]:
hidden_layer_size = 12
# Carga los pesos en las variables Theta1 y Theta2
pesos = {}
pesos['Theta1'] = np.random.rand(hidden_layer_size, input_layer_size + 1)
pesos['Theta2'] = np.random.rand(num_labels, hidden_layer_size + 1)

Theta1, Theta2 = pesos['Theta1'], pesos['Theta2']

# Desenrollar parámetros
print(Theta1.ravel().shape)
print(Theta2.ravel().shape)

nn_params = np.concatenate([Theta1.ravel(), Theta2.ravel()])
print(nn_params.shape)

(720,)
(39,)
(759,)


Se definen funciones como sigmoid y sigmoidGradient para calcular la función sigmoide y su gradiente

In [23]:
def sigmoid(z):
    """
    Computes the sigmoid of z.
    """
    return 1.0 / (1.0 + np.exp(-z))


def sigmoidGradient(z):

    g = np.zeros(z.shape)

    g = sigmoid(z) * (1 - sigmoid(z))

    return g

In [24]:
def nnCostFunction(nn_params,
                   input_layer_size,
                   hidden_layer_size,
                   num_labels,
                   X, y, lambda_= 0.0):

   # Reshape nn_params back into the parameters Theta1 and Theta2, the weight matrices
    # for our 2 layer neural network
    Theta1 = np.reshape(nn_params[:hidden_layer_size * (input_layer_size + 1)],
                        (hidden_layer_size, (input_layer_size + 1)))

    Theta2 = np.reshape(nn_params[(hidden_layer_size * (input_layer_size + 1)):],
                        (num_labels, (hidden_layer_size + 1)))

    m = y.size  # Número de ejemplos de entrenamiento

    J = 0  # Inicializar el costo
    Theta1_grad = np.zeros(Theta1.shape)  # Inicializar el gradiente de Theta1
    Theta2_grad = np.zeros(Theta2.shape)  # Inicializar el gradiente de Theta2

    # Añadir una columna de unos a X (a1 es la entrada a la red)
    a1 = np.concatenate([np.ones((m, 1)), X], axis=1)

    # ----- Comienza el Forward Propagation -----

    # Calcular las activaciones de la capa oculta (a2)
    z2 = a1.dot(Theta1.T)
    a2 = sigmoid(z2)
    a2 = np.concatenate([np.ones((a2.shape[0], 1)), a2], axis=1)  # Añadir una columna de unos a a2

    # Calcular las activaciones de la capa de salida (a3)
    z3 = a2.dot(Theta2.T)
    a3 = sigmoid(z3)

    # Convertir y en una matriz de etiquetas binarias (one-hot encoding)
    y_matrix = np.eye(num_labels)[y]

    # Calcular el término de regularización (excluyendo el sesgo de la primera columna)
    reg_term = (lambda_ / (2 * m)) * (np.sum(np.square(Theta1[:, 1:])) + np.sum(np.square(Theta2[:, 1:])))

    # Calcular la función de costo (J) incluyendo el término de regularización
    J = (-1 / m) * np.sum((np.log(a3) * y_matrix) + np.log(1 - a3) * (1 - y_matrix)) + reg_term

    # ----- Termina el Forward Propagation -----

    # ----- Comienza el Backward Propagation -----

    # Calcular el error en la capa de salida (delta_3)
    delta_3 = a3 - y_matrix

    # Calcular el error en la capa oculta (delta_2) utilizando el gradiente de la función sigmoidea
    delta_2 = delta_3.dot(Theta2[:, 1:]) * sigmoidGradient(z2)

    # Acumular los gradientes
    Delta1 = delta_2.T.dot(a1)
    Delta2 = delta_3.T.dot(a2)

    # Calcular el gradiente regularizado
    Theta1_grad = (1 / m) * Delta1
    Theta1_grad[:, 1:] += (lambda_ / m) * Theta1[:, 1:]  # Regularizar Theta1 (excluyendo el sesgo)
    Theta2_grad = (1 / m) * Delta2
    Theta2_grad[:, 1:] += (lambda_ / m) * Theta2[:, 1:]  # Regularizar Theta2 (excluyendo el sesgo)

    # ----- Termina el Backward Propagation -----

    # Convertir los gradientes en un único vector
    grad = np.concatenate([Theta1_grad.ravel(), Theta2_grad.ravel()])

    return J, grad

In [ ]:
lambda_ = 500
J, _ = nnCostFunction(nn_params, input_layer_size, hidden_layer_size, num_labels, X, y, lambda_)
print('Costo en parametros: %.6f ' % J)

Costo en parametros: 9.481483 


In [25]:
def randInitializeWeights(L_in, L_out, epsilon_init=0.12):
    """
    Randomly initialize the weights of a layer in a neural network.

    Parameters
    ----------
    L_in : int
        Number of incomming connections.

    L_out : int
        Number of outgoing connections.

    epsilon_init : float, optional
        Range of values which the weight can take from a uniform
        distribution.

    Returns
    -------
    W : array_like
        The weight initialiatized to random values.  Note that W should
        be set to a matrix of size(L_out, 1 + L_in) as
        the first column of W handles the "bias" terms."""


    W = np.zeros((L_out, 1 + L_in))
    W = np.random.rand(L_out, 1 + L_in) * 2 * epsilon_init - epsilon_init

    return W

In [26]:
print('Inicialización de parámetros de redes neuronales...')

initial_Theta1 = randInitializeWeights(input_layer_size, hidden_layer_size)
initial_Theta2 = randInitializeWeights(hidden_layer_size, num_labels)

# Desenrrollr parametros
initial_nn_params = np.concatenate([initial_Theta1.ravel(), initial_Theta2.ravel()], axis=0)

Inicialización de parámetros de redes neuronales...


In [27]:
# After you have completed the assignment, change the maxiter to a larger value to see how more training helps.
options = {'maxiter': 10000}

# You should also try different values of lambda
lambda_ = 200

# Create "short hand" for the cost function to be minimized
costFunction = lambda p: nnCostFunction(p, input_layer_size,
                                        hidden_layer_size,
                                        num_labels, X, y, lambda_)

# Now, costFunction is a function that takes in only one argument (the neural network parameters)
res = optimize.minimize(costFunction,
                        initial_nn_params,
                        jac=True,
                        method='L-BFGS-B',  # Cambio a otro método de optimización
                        options=options)

# Get the solution of the optimization
nn_params = res.x

# Obtain Theta1 and Theta2 back from nn_params
Theta1 = np.reshape(nn_params[:hidden_layer_size * (input_layer_size + 1)],
                    (hidden_layer_size, (input_layer_size + 1)))

Theta2 = np.reshape(nn_params[(hidden_layer_size * (input_layer_size + 1)):],
                    (num_labels, (hidden_layer_size + 1)))

In [28]:
def predict(Theta1, Theta2, X):
    """
    Predict the label of an input given a trained neural network
    Outputs the predicted label of X given the trained weights of a neural
    network(Theta1, Theta2)
    """
    # Useful values
    m = X.shape[0]
    num_labels = Theta2.shape[0]

    # You need to return the following variables correctly
    p = np.zeros(m)
    h1 = sigmoid(np.dot(np.concatenate([np.ones((m, 1)), X], axis=1), Theta1.T))
    h2 = sigmoid(np.dot(np.concatenate([np.ones((m, 1)), h1], axis=1), Theta2.T))
    p = np.argmax(h2, axis=1)
    return p

In [29]:
pred = predict(Theta1, Theta2, X[:,:])
print(pred)
print('Training Set Accuracy: %f' % (np.mean(pred == y[:]) * 100))

[0 0 0 ... 0 0 0]
Training Set Accuracy: 58.316254


In [30]:
pred = predict(Theta1, Theta2, Xp[:,:])
print(pred)
print('Testing Set Accuracy: %f' % (np.mean(pred == yp[:]) * 100))


[2 0 2 ... 0 0 0]
Testing Set Accuracy: 58.141001


Después de realizar pruebas y manipular los siguientes parámetros para mejorar la precisión del modelo:

Lambda (regularización de la función de costo para evitar sobreajustes).
Maxiter (número de iteraciones que realizará el optimizer minimizer para una mejor convergencia).
Hidden_layer_size (cantidad de perceptrones en la única capa intermedia.).

Se logró alcanzar precisiones desde el 40% como el más bajo, hasta un máximo del 58%. Aunque esta precisión no sea muy alta ni la esperada, puede deberse a que el dataset de popularidad no muestra una linealidad o una relación muy marcada entre la popularidad de un artículo y sus características. No obstante, se pudo aplicar correctamente una red neuronal adaptándola a una clasificación utilizando la función sigmoide y su gradiente.

Es importante mencionar que los valores de los hiperparámetros utilizados en el programa son aquellos con los que se obtuvo la mayor precisión según las pruebas realizadas. Se observó que la precisión aumenta si no se aplica regularización (lambda=0), pero es crucial aplicar regularización para evitar el sobreajuste. Aunque esto implique una ligera disminución en la precisión, se obtiene un modelo bien generalizado que puede ser más útil en escenarios del mundo real.